In [1]:
# Data processing modules
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("final_train_data.csv", index_col=False)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,13
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,13
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,13
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,13
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,13


In [3]:
labels = data.loc[:, ['784']]
features = data.loc[:, '0':'783']
features.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Model specific imports
import keras
from keras.models import Sequential, model_from_json
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import to_categorical

In [5]:
# Convert labels to cateorical vars
num_classes = len(labels['784'].unique()) + 1 # 1 for zero-based indexing
categorical_y = to_categorical(np.array(labels), num_classes=num_classes)
categorical_y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0.], dtype=float32)

In [6]:
rows = len(data)
X_train = []

for i in range(rows):
    X_train.append(np.array(features[i:i+1]).reshape(1, 28, 28))

X_train = np.array(X_train)
X_train = np.reshape(X_train, (-1, 28, 28, 1))

In [7]:
# Sequential CNN Model
first_conv_num_filters = 30
first_conv_filter_size = 5
second_conv_num_filters = 15
second_conv_filter_size = 3
pool_size = 2

model = Sequential()
model.add(Conv2D(first_conv_num_filters, first_conv_filter_size, input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Conv2D(second_conv_num_filters, second_conv_filter_size, activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
%%time
# Training the model
model.fit(X_train, categorical_y, epochs=10, batch_size=200, shuffle=True, verbose=1)

Epoch 1/10
288/288 [==============================] - 24s 80ms/step - loss: 2.9301 - accuracy: 0.3705
Epoch 2/10
288/288 [==============================] - 23s 80ms/step - loss: 0.4375 - accuracy: 0.8720
Epoch 3/10
288/288 [==============================] - 23s 80ms/step - loss: 0.2652 - accuracy: 0.9203
Epoch 4/10
288/288 [==============================] - 23s 80ms/step - loss: 0.1959 - accuracy: 0.9407
Epoch 5/10
288/288 [==============================] - 23s 80ms/step - loss: 0.1514 - accuracy: 0.9535
Epoch 6/10
288/288 [==============================] - 24s 83ms/step - loss: 0.1263 - accuracy: 0.9607
Epoch 7/10
288/288 [==============================] - 22s 77ms/step - loss: 0.1110 - accuracy: 0.9669
Epoch 8/10
288/288 [==============================] - 26s 92ms/step - loss: 0.1003 - accuracy: 0.9682
Epoch 9/10
288/288 [==============================] - 27s 94ms/step - loss: 0.0871 - accuracy: 0.9732
Epoch 10/10
288/288 [==============================] - 27s 94ms/step - loss: 0.086

In [9]:
# Save model and all the learnt weights
model_json = model.to_json()

with open("model.json", "w") as m:
    m.write(model_json)
    
model.save_weights("weights.h5")    